# V. Accessing SEN-2 and SEN-1 for MOSAIC NLHI Project
---
**Author:** Pierre Sosnowski

---

## 1. Summary

### Satellite data needed
* Sentinel-2 data Level-2A with less than 1% cloud cover, pre-processed using Sen2Cor over periods 01/01/2016-31/12/2017 and 01/01/2020-31/12/2021
* Sentinel-1 data (VV and VH) preprocessed using S1TBX over periods 01/01/2016-31/12/2017 and 01/01/2020-31/12/2021
### Vector data needed
* Polygons of study sites

In [ ]:
# STAC access
import teledetection as tld
import pystac_client

# dataframes
import pandas as pd

# xarrays
import xarray as xry

# library for turning STAC objects into xarrays
import stackstac

# visualization
from matplotlib import pyplot as plt

# miscellanous
import numpy as np
from IPython.display import display
from datetime import date

import matplotlib.dates as mdates

## 2. Which collections exist?

In [ ]:
import pystac_client

# Open the API as before
api = pystac_client.Client.open(
    "https://stacapi-cdos.apps.okd.crocc.meso.umontpellier.fr",
    modifier=tld.sign_inplace,
)

# Get all collections
collections = api.get_collections()

# Print collection IDs
for collection in collections:
    print(collection.id)


## 3. Accessing SEN-2

### 3.1. Request on a STAC catalog

Identify the STAC Collection we're interested in (see [this notebook](01-STAC.ipynb) for a refresher), and load the area of interest defined as a bounding box.

In [ ]:
# Import area of interest
import geopandas as gpd
from shapely.geometry import box

# Define the path to your shapefile
shapefile_path = r"C:\Users\pedro\Dropbox\Database\MOSAIC\GIS\StudyAreas_Brazil_5880.shp"

# Load the shapefile
gdf = gpd.read_file(shapefile_path)

# Display the first few rows
print(gdf.head())

# Reproject to EPSG:4326 if necessary
gdf_wgs84 = gdf.to_crs("EPSG:4326")
bboxgdf = list(gdf_wgs84.total_bounds)

# Get the geometry of the polygon i
first_geom = gdf_wgs84.geometry.iloc[0]

# Get its bounding box as (minx, miny, maxx, maxy)
minx, miny, maxx, maxy = first_geom.bounds

# Optionally, convert it to a list (useful for bbox parameters)
bbox_polyi = [minx, miny, maxx, maxy]

# Print the bounding box
print(f"Bounding box of ith polygon: {bbox_polyi}")


## Display polygons of interest

In [ ]:
# Project polygons on google earth sat map
import folium

# Load shapefile
shapefile_path = r"C:\Users\pedro\Dropbox\Database\MOSAIC\GIS\StudyAreas_Brazil_5880.shp"
gdf = gpd.read_file(shapefile_path)

# Reproject to WGS84 (lat/lon)
gdf_wgs84 = gdf.to_crs(epsg=4326)

# Create a Folium map (initial location doesn’t matter if we fit bounds)
m = folium.Map(tiles="OpenStreetMap")

# Add polygons
for _, row in gdf_wgs84.iterrows():
    # Polygon geometry as GeoJSON
    sim_geo = gpd.GeoSeries([row["geometry"]]).to_json()
    
    # Add polygon
    folium.GeoJson(
        sim_geo,
        style_function=lambda x: {"fillColor": "none", "color": "red", "weight": 2}
    ).add_to(m)
    
    # Compute centroid for label
    centroid = row["geometry"].centroid
    folium.map.Marker(
        [centroid.y, centroid.x],
        icon=folium.DivIcon(
            html=f"""<div style="font-size: 12pt; color: red; font-weight:bold">{row['id']}</div>"""
        )
    ).add_to(m)

# Add Google Satellite basemap
folium.TileLayer(
    tiles="https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
    attr="Google",
    name="Google Satellite",
    overlay=False,
    control=True
).add_to(m)

# Fit map to polygon bounds
m.fit_bounds([[gdf_wgs84.total_bounds[1], gdf_wgs84.total_bounds[0]],
              [gdf_wgs84.total_bounds[3], gdf_wgs84.total_bounds[2]]])

folium.LayerControl().add_to(m)

m


In [ ]:

# retrieving the relevant STAC Item
api = pystac_client.Client.open(
    "https://stacapi-cdos.apps.okd.crocc.meso.umontpellier.fr",
    modifier=tld.sign_inplace,
)

today = date.today()
last_year = today.replace(year=today.year - 1).strftime("%Y-%m")
time_range = f"2016-01-01/2021-12-31"
search = api.search(
    collections=["sentinel2-l2a-sen2cor"], datetime=time_range, bbox=bbox_polyi
)
items = search.item_collection()
print(f"{len(items)} items found")

### 3.2. Explore the items

In [ ]:
items[0]

#### Filter the items based on cloud cover

In [ ]:
item = items[0]
cloud_cover = item.properties.get("eo:cloud_cover")
print(cloud_cover)


In [ ]:
filtered_items = [item for item in items 
                  if item.properties.get("eo:cloud_cover", 100) <= 100]
for item in filtered_items[1:10]:
    print(item.id, item.properties["eo:cloud_cover"])

#### Print asset description of first item

In [ ]:
item = items[0]
assets = [asset.title for asset in item.assets.values()]
descriptions = pd.DataFrame(
    assets,
    columns=["Description"],
    index=pd.Series(item.assets.keys(), name="asset_key"),
)
descriptions

In [ ]:
filtered_items[0]

In [ ]:
len(filtered_items)

### Display filtered images

In [ ]:
import folium

for item in filtered_items:
    # GeoJSON polygon coordinates
    coords = item.geometry['coordinates'][0]  # list of [lon, lat] pairs
    
    # Convert to folium order [lat, lon]
    polygon = [[lat, lon] for lon, lat in coords]
    
    folium.Polygon(
        locations=polygon,
        color='blue',
        weight=1,
        fill=False,
        fill_opacity=0.3,
        popup=item.properties.get('s2:mgrs_tile', 'Tile')
    ).add_to(m)


In [ ]:
from IPython.display import display
display(m)

In [ ]:
m.save("map.html")

#### Convert filtered list to data array

> <span style='color:red'> **IMPORTANT:** The URLs that we obtain from the STAC Catalog are not valid indefinitely. They expire after about 30 minutes. If you see an error in the notebook, it is likely because the URL that you obtained by running the first few cells has now expired.</span> If that happens you should be able to just run the notebook again from the top to get a new URL. You can get longer-lasting URLs by signing up for Planetary Computer (which is free at the time of writing this notebook). More info [here](https://planetarycomputer.microsoft.com/docs/concepts/sas/). 

In [ ]:
bands = ["B02", "B03", "B04", "B05", "B06", "B07", "B08", "B11", "B12"]
FILL_VALUE = 2**16 - 1
array = stackstac.stack(
    filtered_items,
    assets=bands,
    resolution=10,
    fill_value=FILL_VALUE,
    bounds_latlon=bbox_polyi,
    epsg = 4326,
)
array

In [ ]:
 # counting the number of acquisitons per month
import matplotlib.dates as mdates

# resampling and counting for each month
# using MS (Month Start) as we prefer using the first day
# of the month as a reference point instead of the last
# acquisition date of the month
ar = array.time.sortby("time").resample(time="MS").count()


fig, ax = plt.subplots(figsize=(35, 13))
ar.plot.scatter(ax=ax, x="time")
ax.set_ylabel("Acquisition count")
ax.set_title("Number of acquisitions per month")

# setting y ticks manually to avoid non-integer values being used
ymin = int(ar.min().values)
ymax = int(ar.max().values)

ax.set_yticks(range(ymin, ymax + 1,5))

# Format x-axis to show year-month
ax.xaxis.set_minor_locator(mdates.MonthLocator())  
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))  

fig.autofmt_xdate()  # rotate labels so they don't overlap

# Grid on minor ticks (months)
ax.grid(which="minor", linestyle="--", alpha=0.5)
ax.grid(which="major", linestyle="-", alpha=0.8)

fig.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Convert coordinates to a DataFrame
df = array.coords.to_dataset()[["s2:mgrs_tile"]].to_dataframe()

# Add month column (month start)
df['month'] = df.index.to_period('M').to_timestamp()

# Count number of acquisitions per tile per month
counts = df.groupby(['month', 's2:mgrs_tile']).size().unstack(fill_value=0)

counts

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import math

# Number of tiles
tiles = counts.columns
n_tiles = len(tiles)

# Determine grid size
n_cols = 2  # number of columns in the subplot grid
n_rows = math.ceil(n_tiles / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(8*n_cols, 3*n_rows), sharex=True, sharey=True)
axes = axes.flatten()  # flatten in case of 2D array

for i, tile in enumerate(tiles):
    ax = axes[i]
    ax.scatter(counts.index, counts[tile], color='tab:blue', s=50, marker = "+")
    ax.set_title(tile)
    ax.tick_params(axis='x', rotation=45)
    # Format x-axis to show year-month
    ax.xaxis.set_minor_locator(mdates.MonthLocator())  
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))  
        
    # Grid on minor ticks (months)
    ax.grid(which="minor", linestyle="--", alpha=0.5)
    ax.grid(which="major", linestyle="-", alpha=0.8)


# Remove any empty subplots
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

fig.suptitle("Monthly Sentinel-2 acquisitions per tile", fontsize=16)
fig.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


### Number of SEN2 images over each polygon

In [ ]:
import geopandas as gpd
from shapely.geometry import shape
import pandas as pd

# Convert filtered_items (from stackstac search or STAC API) to GeoDataFrame
geoms = [shape(item.geometry) for item in filtered_items]
gdf_images = gpd.GeoDataFrame(pd.DataFrame(filtered_items), geometry=geoms, crs="EPSG:4326")

# Spatial join to find which images overlap each AOI polygon
join = gpd.sjoin(gdf_images, gdf_wgs84, predicate="intersects", how="inner")

# Count number of images per AOI polygon
image_counts = join.groupby("index_right").size()

# Add results back to the AOI shapefile
gdf["n_images"] = gdf.index.map(image_counts).fillna(0).astype(int)

print("✅ Number of SEN2 overlapping images computed for each polygon")

# Optional: visualize counts
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(8, 8))
gdf.plot(column="n_images", cmap="YlGnBu", legend=True, edgecolor="black", ax=ax)
ax.set_title("Number of Sentinel-2 Images Overlapping Each AOI", fontsize=12)
ax.axis("off")
plt.show()


### Display SEN2 images

## 4. Access SEN-1

In [ ]:
# retrieving the relevant STAC Item
api = pystac_client.Client.open(
    "https://stacapi-cdos.apps.okd.crocc.meso.umontpellier.fr",
    modifier=tld.sign_inplace,
)

today = date.today()
last_year = today.replace(year=today.year - 1).strftime("%Y-%m")
time_range = f"2016-01-01/2021-12-31"
search = api.search(
    collections=["s1-grd-sigma0-ortho"], datetime=time_range, bbox=bbox_polyi
)
sen1items = search.item_collection()
print(f"{len(sen1items)} items found")

In [ ]:
item = sen1items[0]
assets = [asset.description for asset in item.assets.values()]
descriptions = pd.DataFrame(
    assets,
    columns=["Description"],
    index=pd.Series(item.assets.keys(), name="asset_key"),
)
pd.set_option('display.max_colwidth', 200)
descriptions

In [ ]:
sen1items[0]

### Number of SEN1 images over each polygon

In [ ]:
import geopandas as gpd
from shapely.geometry import shape
import pandas as pd

# Convert filtered_items (from stackstac search or STAC API) to GeoDataFrame
geoms = [shape(item.geometry) for item in sen1items]
gdf_images = gpd.GeoDataFrame(pd.DataFrame(sen1items), geometry=geoms, crs="EPSG:4326")

# Spatial join to find which images overlap each AOI polygon
join = gpd.sjoin(gdf_images, gdf_wgs84, predicate="intersects", how="inner")

# Count number of images per AOI polygon
image_counts = join.groupby("index_right").size()

# Add results back to the AOI shapefile
gdf["n_images"] = gdf.index.map(image_counts).fillna(0).astype(int)

print("✅ Number of overlapping SEN1 images computed for each polygon")

# Optional: visualize counts
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(8, 8))
gdf.plot(column="n_images", cmap="YlGnBu", legend=True, edgecolor="black", ax=ax)
ax.set_title("Number of Sentinel-1 Images Overlapping Each AOI", fontsize=12)
ax.axis("off")
plt.show()


### SEN-1 acquisitions over time

In [ ]:
bands = ["QL","VH","VV"]
FILL_VALUE = 2**16 - 1
sen1array = stackstac.stack(
    sen1items,
    assets=bands,
    resolution=10,
    fill_value=FILL_VALUE,
    bounds_latlon=bbox_polyi,
    epsg = 4326,
)
sen1array

In [ ]:
 # counting the number of acquisitons per month
import matplotlib.dates as mdates

# resampling and counting for each month
# using MS (Month Start) as we prefer using the first day
# of the month as a reference point instead of the last
# acquisition date of the month
s1ar = sen1array.time.sortby("time").resample(time="MS").count()


fig, ax = plt.subplots(figsize=(35, 13))
s1ar.plot.scatter(ax=ax, x="time")
ax.set_ylabel("Acquisition count")
ax.set_title("Number of acquisitions per month")

# setting y ticks manually to avoid non-integer values being used
ymin = int(s1ar.min().values)
ymax = int(s1ar.max().values)

ax.set_yticks(range(ymin, ymax + 1,5))

# Format x-axis to show year-month
ax.xaxis.set_minor_locator(mdates.MonthLocator())  
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))  

fig.autofmt_xdate()  # rotate labels so they don't overlap

# Grid on minor ticks (months)
ax.grid(which="minor", linestyle="--", alpha=0.5)
ax.grid(which="major", linestyle="-", alpha=0.8)

fig.show()